In [1]:
import torch
import matplotlib.pyplot as plt
import torch

from utils.TD import TD_SGD, TD_Adam
from utils.TD_NN import TD_NN_Adam, TD_NN_SGD

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import numpy as np

seed = 42
torch.manual_seed(seed)

In [2]:
housedata = torch.tensor(np.loadtxt('data\\readyhousedata.txt', delimiter=','), dtype=torch.float32)

X = housedata[:, :-1]
y = housedata[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

reg = LinearRegression() 
reg.fit(X_train, y_train)

weights = torch.tensor(reg.coef_)
intercept = torch.tensor(reg.intercept_)

print("Weights learned using LR:\n", weights)
print("Intercept:", intercept)

score = reg.score(X_test, y_test)
print("Model R^2 score:", score)

Weights learned using LR:
 tensor([-10.9937,   5.0058,   0.6358,   2.4487, -10.3119,  17.5176,   1.0312,
        -17.7757,   7.3409,  -6.4276,  -8.7997,   3.0356, -20.5194])
Intercept: tensor(29.0443)
Model R^2 score: 0.7730569243431091


In [3]:
num_samples, num_features = X_train.shape
P = torch.ones((num_samples, num_samples)) / num_samples # Equal probability to move to any state

alpha = 0.01  # Learning rate
gamma = 0   # Discount factor
num_iterations = 1e5  # Number of iterations
epsilon = 1e-9

td_sgd = TD_SGD(
    n_iter=num_iterations,
    P=P,
    link=lambda x : x,
    inv_link=lambda x : x,
    gamma=gamma,
    alpha=alpha,
    epsilon=epsilon,
    random_state=seed,
)

td_sgd.fit(X_train, y_train)

w_hat_house = td_sgd.weights
bias_house = td_sgd.bias

print("Weights learned using TD SGD:\n", w_hat_house)

Weights learned using TD SGD:
 tensor([-1.0654e+01,  4.9181e+00,  9.4121e-03,  1.4318e+00, -1.0087e+01,
         1.7143e+01,  8.6331e-01, -1.7416e+01,  7.2050e+00, -6.5871e+00,
        -8.0881e+00,  2.8847e+00, -2.0169e+01])


In [4]:
alpha = 0.05  # Learning rate
gamma = 0   # Discount factor
num_iterations = 1e5  # Number of iterations
epsilon = 1e-9

td_adam = TD_Adam(
    n_iter=num_iterations,
    P=P,
    link=lambda x : x,
    inv_link=lambda x : x,
    gamma=gamma,
    alpha=alpha,
    epsilon=epsilon,
    random_state=seed,
)

td_adam.fit(X_train, y_train)

print("Weights learned using TD Adam:\n", td_adam.weights)

Ending optimization early at iteration 85489
Weights learned using TD Adam:
 tensor([-12.1528,   4.4889,   0.4160,   1.6978,  -9.6249,  16.9506,   1.4479,
        -18.0856,   7.0949,  -6.5179,  -8.8375,   2.6827, -21.3809])


In [24]:
# Initialize the TD_NN_SGD model
td_nn_sgd = TD_NN_Adam(
    n_iter=1e5,
    P=P,
    link=lambda x : x,
    inv_link=lambda x : x,
    gamma=0,
    alpha=0.001,
    epsilon=1e-6,
    input_size=num_features,
)

# Fit the model
td_nn_sgd.fit(X_train, y_train)

# Evaluate the model
rmse = td_nn_sgd.rmse(X_test, y_test)
print(f"RMSE on test set: {rmse:.4f}")

Ending optimization early at iteration 6848
RMSE on test set: 3.1640


In [6]:
pred_TD_sgd = td_sgd.predict(X_test)
pred_TD_adam = td_adam.predict(X_test)
pred_L2 = reg.predict(X_test)

rmse_TD_sgd = td_sgd.rmse(X_test, y_test)
rmse_TD_adam = td_adam.rmse(X_test, y_test)
rmse_L2 = torch.sqrt(torch.tensor(mean_squared_error(y_test, pred_L2)))

print(f"RMSE on the test set using TD SGD: {rmse_TD_sgd}")
print(f"RMSE on the test set using TD Adam: {rmse_TD_adam}")
print(f"RMSE on the test set using L2 Regression: {rmse_L2}")
print("---------------")
print(f"Norm of difference in weights for L2 and TD SGD: {torch.norm(weights - td_sgd.weights, 2)}")
print(f"Norm of difference in weights for L2 and TD Adam: {torch.norm(weights - td_adam.weights, 2)}")


RMSE on the test set using TD SGD: 4.25275182723999
RMSE on the test set using TD Adam: 4.218838214874268
RMSE on the test set using L2 Regression: 4.154654026031494
---------------
Norm of difference in weights for L2 and TD SGD: 1.6105726957321167
Norm of difference in weights for L2 and TD Adam: 2.0549962520599365
